<a href="https://colab.research.google.com/github/devanshugupta/AbstractArt_GAN/blob/main/ArtGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget web.fsktm.um.edu.my/~cschan/source/ICIP2017/wikiart.zip

--2021-02-18 17:17:00--  http://web.fsktm.um.edu.my/~cschan/source/ICIP2017/wikiart.zip
Resolving web.fsktm.um.edu.my (web.fsktm.um.edu.my)... 103.18.2.145
Connecting to web.fsktm.um.edu.my (web.fsktm.um.edu.my)|103.18.2.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27301958850 (25G) [application/zip]
Saving to: ‘wikiart.zip’

wikiart.zip         100%[===================>]  25.43G  7.48MB/s    in 61m 19s 

2021-02-18 18:18:19 (7.08 MB/s) - ‘wikiart.zip’ saved [27301958850/27301958850]



In [ ]:
#!unzip wikiart.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!cp -r /content/wikiart/Cubism /content/drive/MyDrive/dataset

In [ ]:
!ls /content/drive/MyDrive/wikiart/High_Renaissance | wc -l


1343


In [ ]:
## image_resizer.py
# Importing required libraries
import os
import numpy as np
from PIL import Image

# Defining an image size and image channel
# We are going to resize all our images to 128X128 size and since our images are colored images
# We are setting our image channels to 3 (RGB)

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = '/content/drive/MyDrive/wikiart/Minimalism'
# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR 

training_data = []
print('resizing...')

for file in [,'/content/drive/MyDrive/wikiart/Cubism','/content/drive/MyDrive/wikiart/High_Renaissance']:
    count=0
    for filename in os.listdir(file):
        path = os.path.join(file, filename)
        image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

        training_data.append(np.asarray(image))
        count+=1
        print(count,end=',')
    print('')
training_data = np.reshape(
    training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
training_data = training_data / 127.5 - 1
    
print('saving file...')
np.save('MCH_data.npy', training_data)

resizing...
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,

In [ ]:
!cp -r /content/MCH_data.npy /content/drive/MyDrive/

In [ ]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os
import pickle

In [ ]:
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4
SAVE_FREQ = 100
# Size vector to generate images from
NOISE_SIZE = 100
# Configuration
EPOCHS = 10000 # number of iterations
BATCH_SIZE = 32
GENERATE_RES = 3
IMAGE_SIZE = 128 # rows/cols
IMAGE_CHANNELS = 3

In [ ]:
training_data = np.load('/content/drive/MyDrive/MCH_data.npy')

In [ ]:
training_data.shape

(4915, 128, 128, 3)

In [ ]:
def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2,input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [ ]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu', input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    
    return Model(input, generated_image)

In [ ]:
def save_images(cnt, noise):
    image_array = np.full((PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
    output_path = 'outputAll'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [ ]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = Adam(1.5e-4, 0.5)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    x_real = training_data[idx]
    
    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)
    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)
    
    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    
    generator_metric = combined.train_on_batch(noise, y_real)
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise)
        cnt += 1
        print(epoch,'epoch', 'Discriminator accuracy:', 100*  discriminator_metric[1], 
        'Generator accuracy:', 100 * generator_metric[1])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 4096)              413696    
_________________________________________________________________
reshape_6 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_34 (UpSampling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_62 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_40 (Activation)   (None, 8, 8, 256)         0         
_________________________________________________________________
up_sampling2d_35 (UpSampling (None, 16, 16, 256)     

In [ ]:
!cp -r /content/outputAll /content/drive/MyDrive/outputAll

In [ ]:
discriminator.save('/content/drive/MyDrive/outputAll/discriminator.h5')
generator.save('/content/drive/MyDrive/outputAll/generator.h5')
combined.save('/content/drive/MyDrive/outputAll/combined.h5')

In [ ]:
loaded_model = load_model('/content/drive/MyDrive/outputAll/combined.h5')
result = loaded_model.summary()
print(result)

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
model_13 (Functional)        (None, 128, 128, 3)       3376131   
_________________________________________________________________
model_12 (Functional)        (None, 1)                 1720385   
Total params: 5,096,516
Trainable params: 5,092,036
Non-trainable params: 4,480
_________________________________________________________________
None


In [ ]:
!ls /content/drive/MyDrive/outputAll/ | wc -l

6
